In [2]:
// declare our variables up here
let prompt, files, issue, i, examples;
const OPENAI_API_URL = 'https://api.openai.com/v1/engines/code-davinci-001/completions';


undefined

In [3]:
issue = `@file1:/path/to/file1
@file2:/path/to/file2

The amount of CPU in @file1 needs to be increased to 512M
The PVC storage amount in @file2 should be decreased to 10Gi
`;

'@file1:/path/to/file1\n' +
  '@file2:/path/to/file2\n' +
  '\n' +
  'The amount of CPU in @file1 needs to be increased to 512M\n' +
  'The PVC storage amount in @file2 should be decreased to 10Gi\n'

## Reading File Data
We'll use an `examples` directory to describe our scenarios. Every scenario consists of a directory called `files` containing the initial files that we'll be referencing, and a `issues` directory consisting of subdirectories named by the issue numbers, e.g. `issues/1`, `issues/2`, etc. Under each subdirectory, there is an `issue.md` file describing the changes to be made, and a `completions` directory where the files will be written to after being updated.


For example:

```
files/
	README.md
	index.js
	package.json
	package-lock.json
issues/
	1/
		issue.md
		completions/
			index.js
			package.json
	2/
		issue.md
		completions/
			README.md
			package-lock.json
	3/
		issue.md
		completions/
			README.md
			index.js
			package.json
```



In [4]:
// import libraries
const path = require('path');
const fs = require('fs');

undefined

In [5]:
// list out the contents of the current directory
let localFiles = fs.readdirSync('./examples');

undefined

In [6]:
// return a map of {fileName => {path: path, content: content}}
const getFilesFromIssue = (issue) => {
	// extract a map of the files from the issue based on the following regex:
	// /`(@([a-zA-Z0-9_\-]+):(.+))`/g
	const fileRegex = /`(@([a-zA-Z0-9_\-]+):(.+))`/g;
	// create a map from the filename to the filepath
	const fileMap = new Map();
	// extract the string from group 3 of the regex
	let match;
	while (match = fileRegex.exec(issue)) {
		let [name, path] = [match[2], match[3]];
		if (!fileMap.has(name)) {
			// define the file object here 
			fileMap.set(name, {
				path: path,
				content: '',
				updatedContent: '',
			});
		} else {
			console.error(`duplicate file name ${name}`);
		}
	}
	return fileMap;
}


undefined

In [7]:
let getFilenamesFromIssue;

undefined

In [8]:
const populateFiles = (fileMap, rootDir) => {
	// look through the fileMap and read the contents of the file at the given path
	for (const [_, file] of fileMap) {
		let searchPath = path.join(rootDir, file.path);
		file.content = fs.readFileSync(searchPath, 'utf8');
	}
	return fileMap;
}

undefined

In [9]:
let getIssuesForDirectory;

undefined

In [10]:
getIssuesForDirectory = (dir) => {
	/* return a list of issue objects of the form: 
		{
			relevantFiles: string[],
			content: string,
			issueNumber: number
		}
	*/ 
	let issues = fs.readdirSync(dir).map((issueNo) => {
		let issuePath = path.join(dir, issueNo);
		let issue = fs.readFileSync(path.join(issuePath, 'issue.md'), 'utf8');
		return {
			relevantFiles: getFilesFromIssue(issue),
			content: issue,
			// convert the issueNo to a number
			issueNumber: parseInt(issueNo)
		};
	});
	// sort the issues by issue number
	issues.sort((a, b) => a.issueNumber - b.issueNumber);
	return issues;
}

[Function: getIssuesForDirectory]

In [11]:
// call getIssuesForDirectory with the current directory
console.log(getIssuesForDirectory('./examples/simple-prometheus-update/issues'));
console.log(getIssuesForDirectory('./examples/simple-prometheus-update/issues')[0].relevantFiles);

[
  {
    relevantFiles: Map(2) { 'file1' => [Object], 'file2' => [Object] },
    content: '`@file1:grafana/base/datasource.yaml`\n' +
      '`@file2:grafana/base/grafana-route.yaml`\n' +
      '\n' +
      '@file1 and @file2 must specify the `grafana-datasource` namespace.\n' +
      "@file1 needs to increase the 'Prometheus' data source to a time interval of 20s.\n" +
      'Disable tls-acme for @file2.\n',
    issueNumber: 1
  }
]
Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: '',
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: '',
    updatedContent: ''
  }
}


undefined

In [12]:
examples = new Map();
// loop through local files in ./examples
// examples is a directory 
for (const dirname of fs.readdirSync('./examples')) {
	console.log(dirname);
	examples.set(dirname, {
		issues: [],	// list of issues, sorted by their number
	})

	let issueDir = path.join('./examples', dirname, 'issues');
	examples.get(dirname).issues = getIssuesForDirectory(issueDir);
	if (examples.get(dirname).issues.length === 0) {
		// no issues, so skip this directory
		continue;
	}
	let firstIssue = examples.get(dirname).issues[0];
	console.log("first item in issues", firstIssue);
	// populate the files map
	examples.get(dirname).files = getFilesFromIssue(firstIssue.content);
	populateFiles(examples.get(dirname).files, path.join('./examples', dirname, 'files'));
}
console.log(examples);

simple-prometheus-update
first item in issues {
  relevantFiles: Map(2) {
    'file1' => {
      path: 'grafana/base/datasource.yaml',
      content: '',
      updatedContent: ''
    },
    'file2' => {
      path: 'grafana/base/grafana-route.yaml',
      content: '',
      updatedContent: ''
    }
  },
  content: '`@file1:grafana/base/datasource.yaml`\n' +
    '`@file2:grafana/base/grafana-route.yaml`\n' +
    '\n' +
    '@file1 and @file2 must specify the `grafana-datasource` namespace.\n' +
    "@file1 needs to increase the 'Prometheus' data source to a time interval of 20s.\n" +
    'Disable tls-acme for @file2.\n',
  issueNumber: 1
}
Map(1) {
  'simple-prometheus-update' => {
    issues: [ [Object] ],
    files: Map(2) { 'file1' => [Object], 'file2' => [Object] }
  }
}


undefined

In [13]:
files = new Map([
  ['file1', {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  }],
  ['file2', {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: route.openshift.io/v1\n' +
      'metadata:\n' +
      '  name: grafana\n' +
      '  annotations:\n' +
      '    kubernetes.io/tls-acme: "true"\n' +
      'spec:\n' +
      '  host: grafana.operate-first.cloud\n' +
      '  to:\n' +
      '    kind: Service\n' +
      '    name: grafana-service\n' +
      '  port:\n' +
      '    targetPort: 3000\n',
    updatedContent: ''
  }]
]);

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'\n",
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: ro

## Building the prompt

In [14]:
let buildPrompt;

undefined

In [15]:
// @issue: string
// @files: map of {fileName: string => {path: string, content: string, updatedContent: string}}
buildPrompt = (issue, files) => {
	// read the prefix from 'prefix.md'
	const prefix = fs.readFileSync('./prefix.md', 'utf8');


	let prompt = `${prefix}
## Description of issues:
${issue}

## Original files:
`;

	i = 0;
	for (const [fileName, file] of files) {
		prompt += `# @${fileName}\n${file.content}\n`;
		// only place the delimiting string if in-between files
		if (files.size > 1 && i < files.size - 1) {
			prompt += '---\n';
		}
		i++;
	}

	prompt += `
## Updated files:
`;
	return prompt;
}

[Function: buildPrompt]

In [16]:
// iterate through each example and build the corresponding prompt
for (const [dirname, example] of examples) {
	// build the initialprompt
	let initialPrompt = buildPrompt(example.issues[0].content, example.files);
	// write the prompt to a file
	fs.writeFileSync(path.join('./examples', dirname, 'prompt.md'), initialPrompt);
}

undefined

# Updating Files

We retrieve a completion from OpenAI's completion endpoint and split the files up by a '---' delimiter,
then we'll match them to their corresponding files.

Let's define a few functions to help us with this. We'll bring in the `axios` package to make our HTTP requests.

In [17]:
let completionToFiles, getCompletion;
var axios = require('axios');

undefined

In [18]:
// process the completion & place it into the files' updatedContent field
completionToFiles = (completion, files) => {
	let completions = completion.split('---');
	// go through the list of completions, extract the filename and set the updated content
	for (const cmpltn of completions) {
		// extract the file tag from the completion
		const fileTagRegex = /#\s*\@(.+)/g;
		const match = fileTagRegex.exec(cmpltn);
		if (match !== null) {
			const fileTag = match[1];
			if (files.has(fileTag)) {
				// find the line containing the fileTag and remove all lines up to and including the fileTag 
				const lines = cmpltn.split('\n');
				let i = 0;
				for (const line of lines) {
					i++;
					if (line.includes(fileTag)) {
						break;
					}
				}
				// remove the lines from the completion
				const newCompletion = lines.slice(i).join('\n');
				// set the updated content
				files.get(fileTag).updatedContent = newCompletion;
			};
		}
	}
};

// to create the completion
getCompletion = async (prompt, maxTokens, stopSequences) => {
	stopSequences = stopSequences || ['####',];
	const headers = {
		// get OPENAI_API_KEY from env
		"Authorization": `Bearer ${process.env.OPENAI_API_KEY}`,
		"Content-Type": "application/json",
	};
	// console.log("headers", headers);
	const body = {
		prompt: prompt,
		max_tokens: maxTokens | 128,
		stop: stopSequences,
		temperature: 0.2,
		top_p: 1,
		frequency_penalty: 0,
		presence_penalty: 0,
	};
	let completion;

	// request the openai api using axios
	await axios.post(OPENAI_API_URL, body, { headers }).then(async (response) => {
		// update the object with the competion result
		if (response.status == 200 && response.data.choices) {
			if (response.data.choices.length > 0) {
				completion = response.data.choices[0].text;
			} else {
				console.error("no completion found");
			}
		}
	});
	return completion;
};



[AsyncFunction: getCompletion]

In [19]:
const updateFilesFromCompletion = async (files, prompt, maxTokens) => {
	const completion = await getCompletion(prompt, maxTokens, ['####',]);
	completionToFiles(completion, files);
}

undefined

In [20]:
let writeFilesToCompletionsDir;

undefined

In [46]:
// filesMap: map of {fileName: string => {path: string, content: string, updatedContent: string}}
// basePath: string
writeFilesToCompletionsDir = (filesMap, basePath) => {
		// delete the completions directory if it exists
		if (fs.existsSync(path.join(basePath, 'completions'))) {
			fs.rmSync(path.join(basePath, 'completions'), { recursive: true });
		}

		// write the updated files into the completions directory using their same path as the original files
		for (const [_, file] of filesMap) {
			let outputPath = path.join(basePath, 'completions', file.path);
			console.log('writing to: ', outputPath);
			fs.mkdirSync(path.dirname(outputPath), { recursive: true });
			// write the file and create parent directories, if needed
			fs.writeFileSync(outputPath, file.updatedContent);
		}
}

[Function: writeFilesToCompletionsDir]

In [22]:
let writeUpdatedContentToFiles;

undefined

In [37]:
writeUpdatedContentToFiles = (filesMap, basePath) => {
	// go through each file and write the updated content to the file
	for (const [filename, file] of filesMap) {
		let outputPath = path.join(basePath, 'files', file.path);
		console.log(`updating file: "${filename}": ${file.path}`);
		fs.writeFileSync(outputPath, file.updatedContent);
	}
}

[Function: writeUpdatedContentToFiles]

In [24]:
let populateFileMap;

undefined

In [25]:
populateFileMap = (filesMap, basePath) => {
	// populate the example's files from the main files
	for (const [_, file] of filesMap) {
		// read the file from the file.path and set the content
		let filePath = path.join(basePath, file.path);
		file.content = fs.readFileSync(filePath, 'utf8');
	}	
}

[Function: populateFileMap]

In [26]:
let generateCompletionsOnExample;

undefined

In [41]:
generateCompletionsOnExample = async (basePath, example) => {
	for (let k = 0; k < example.issues.length; k++) {
		let issue = example.issues[k];
		let issuePath = path.join(basePath, 'issues', example.issues[k].issueNumber.toString());
		console.log(`::: example ${issue.issueNumber} in "${issuePath}" :::`);
		console.log(`\`\`\`\n${issue.content}\n\`\`\`\n`);

		// populate the filesMap with the files from the issue
		populateFileMap(issue.relevantFiles, path.join(basePath, 'files'));

		// build the initial prompt
		let initialPrompt = buildPrompt(example.issues[k].content, issue.relevantFiles);
		// write the prompt to a file
		fs.writeFileSync(path.join(issuePath, 'prompt.md'), initialPrompt);

		// update the files with the completion
		let completion = await getCompletion(initialPrompt, 512, ['####',]);
		
		// if an existing completion.md file exists, delete it
		if (fs.existsSync(path.join(issuePath, 'completion.md'))) {
			fs.unlinkSync(path.join(issuePath, 'completion.md'));
		}

		// write the completion to a file
		fs.writeFileSync(path.join(issuePath, 'completion.md'), [initialPrompt, completion].join());

		completionToFiles(completion, issue.relevantFiles);			
		console.log('files after converting completions', issue.relevantFiles);
		writeFilesToCompletionsDir(issue.relevantFiles, issuePath);

		// update base files with the completion
		writeUpdatedContentToFiles(issue.relevantFiles, basePath);
	}
}

[AsyncFunction: generateCompletionsOnExample]

Now we'll run the file updater

In [28]:
// $$.async();

{
	// iterate through each example in the examples directory and generate completions
	let examples = fs.readdirSync('./examples');
	console.log(examples);
	for (const [dirname, example] of examples) {
		console.log(`===== processing example '${dirname}' ======`);
		// generateCompletionsOnExample(path.join('./examples', dirname), example);
	}
}

[ 'simple-prometheus-update' ]
===== processing example 's' ======


undefined

In [29]:
prompt

undefined

In [30]:
examples.get('simple-prometheus-update').files

Map(2) {
  'file1' => {
    path: 'grafana/base/datasource.yaml',
    content: 'apiVersion: integreatly.org/v1alpha1\n' +
      'kind: GrafanaDataSource\n' +
      'metadata:\n' +
      '  name: datasource\n' +
      'spec:\n' +
      '  name: prometheus-grafanadatasource.yaml\n' +
      '  datasources:\n' +
      '    - name: Prometheus\n' +
      '    - access: proxy\n' +
      '      editable: true\n' +
      '      isDefault: true\n' +
      '      jsonData:\n' +
      "        httpHeaderName1: 'Authorization'\n" +
      '        timeInterval: 5s\n' +
      '        tlsSkipVerify: true\n' +
      '      name: Prometheus\n' +
      '      secureJsonData:\n' +
      "        httpHeaderValue1: 'Bearer ${BEARER_TOKEN}'\n" +
      '      type: prometheus\n' +
      "      url: 'https://thanos-querier.openshift-monitoring.svc.cluster.local:9091'",
    updatedContent: ''
  },
  'file2' => {
    path: 'grafana/base/grafana-route.yaml',
    content: 'kind: Route\n' +
      'apiVersion: rout

In [31]:
// output the full contents of the firt example in the examples map
console.log(examples.get('simple-prometheus-update'))

{
  issues: [
    {
      relevantFiles: [Map],
      content: '`@file1:grafana/base/datasource.yaml`\n' +
        '`@file2:grafana/base/grafana-route.yaml`\n' +
        '\n' +
        '@file1 and @file2 must specify the `grafana-datasource` namespace.\n' +
        "@file1 needs to increase the 'Prometheus' data source to a time interval of 20s.\n" +
        'Disable tls-acme for @file2.\n',
      issueNumber: 1
    }
  ],
  files: Map(2) {
    'file1' => {
      path: 'grafana/base/datasource.yaml',
      content: 'apiVersion: integreatly.org/v1alpha1\n' +
        'kind: GrafanaDataSource\n' +
        'metadata:\n' +
        '  name: datasource\n' +
        'spec:\n' +
        '  name: prometheus-grafanadatasource.yaml\n' +
        '  datasources:\n' +
        '    - name: Prometheus\n' +
        '    - access: proxy\n' +
        '      editable: true\n' +
        '      isDefault: true\n' +
        '      jsonData:\n' +
        "        httpHeaderName1: 'Authorization'\n" +
        '

undefined

## Processing Examples

We will use the following structure to process examples:
1. Read the `examples` directory to get a list of all the example directories
2. For each `example`, do the following:
	1. Gather the list of issues in the `issues` directory
	2. For each issue:
		1. gather the relevant files from the `files` directory
		2. Generate a prompt based on the issue & files
		3. Try and retrieve a completion from OpenAI's completion endpoint:
			- If successful:
				1. Gather the updated files, write their contents to the original files in `files`
				2. Create a copy of the completed files in the `completions` directory 
			- Otherwise, stop processing the issues
	3. Write the last successfully processed issue to a `last-processed` file


In [32]:
let processExample;

undefined

In [42]:
processExample = async (baseDir) => {
	// first check to see if a last-processed.json file exists
	let lastProcessedFile = path.join(baseDir, 'last-processed.json');
	let lastProcessed;
	if (fs.existsSync(lastProcessedFile)) {
		lastProcessed = JSON.parse(fs.readFileSync(lastProcessedFile, 'utf8'));
	} else {
		lastProcessed = {
			issueNumber: 0,
		};
	}

	// load the issues 
	let issues = getIssuesForDirectory(path.join(baseDir, 'issues'));
	
	// we need to process all issues whose number is greater than the last processed issue number
	let issuesToProcess = issues.filter((issue) => issue.issueNumber > lastProcessed.issueNumber);

	try {
		// process each issue until failure or completion
		for (let issue of issuesToProcess) {
			// populate the issue's files 
			populateFileMap(issue.relevantFiles, path.join(baseDir, 'files'));
			
			// generate a prompt from the issue's content and files
			let initialPrompt = buildPrompt(issue.content, issue.relevantFiles);
			
			// write the prompt to a file
			// if an existing prompt.md file exists, delete it
			let issuePath = path.join(baseDir, 'issues', issue.issueNumber.toString());
			if (fs.existsSync(path.join(issuePath, 'prompt.md'))) {
				fs.unlinkSync(path.join(issuePath, 'prompt.md'));
			}
			fs.writeFileSync(path.join(issuePath, 'prompt.md'), initialPrompt);

			// obtain a completion 
			let completion = await getCompletion(initialPrompt, 512, ['####',]);

			// write the completion to a file, if one already exists then overwrite it
			if (fs.existsSync(path.join(issuePath, 'completion.md'))) {
				fs.unlinkSync(path.join(issuePath, 'completion.md'));
			}
			fs.writeFileSync(path.join(issuePath, 'completion.md'), [initialPrompt, completion].join(), );

			// convert the completion to the issue's files
			completionToFiles(completion, issue.relevantFiles);			
			writeFilesToCompletionsDir(issue.relevantFiles, issuePath);
			writeUpdatedContentToFiles(issue.relevantFiles, baseDir);
			
			// update the last processed issue number
			lastProcessed.issueNumber = issue.issueNumber;
		}
	} catch(e) {
		console.log(e);
	} finally {
		// write the last processed issue number to a file
		fs.writeFileSync(lastProcessedFile, JSON.stringify(lastProcessed));
	}
};

[AsyncFunction: processExample]

In [34]:
let processExamples;

undefined

In [43]:
processExamples = async () => {
	const examplesDir = './examples';
	const examples = fs.readdirSync(examplesDir);
	for (const dirname of examples) {
		let exampleDir = path.join(examplesDir, dirname);
		// read the last issue processed from the example directory
		await processExample(exampleDir);
	}
}

[AsyncFunction: processExamples]

In [47]:
$$.async();
{
	processExamples();
}

writing to:  examples/simple-prometheus-update/issues/1/completions/grafana/base/datasource.yaml
writing to:  examples/simple-prometheus-update/issues/1/completions/grafana/base/grafana-route.yaml
updating file: "file1": grafana/base/datasource.yaml
updating file: "file2": grafana/base/grafana-route.yaml
writing to:  examples/simple-prometheus-update/issues/2/completions/grafana/base/datasource.yaml
updating file: "file1": grafana/base/datasource.yaml


undefined